# Init stuffs

### Imports

In [1]:
import operator

### Input data

In [2]:
filename = "./input_1.txt"
test_filename = "./input_2.txt"

with open(file=filename) as f:
    data = f.read().split("\n\n")
    instructions, parts = [data[i].splitlines() for i in range(len(data))]

with open(file=test_filename) as f:
    test_data = f.read().split("\n\n")
    test_instructions, test_parts = [test_data[i].splitlines() for i in range(len(test_data))]

# Pt. 1

In [3]:
OPS = { ">": operator.gt, "<": operator.lt}

In [4]:
def simplify_instruction(instruction: str) -> tuple[str,list[str]]:
    name, instr = instruction.split("{")
    instr = instr.strip("}").split(",")
    instr_dict = {}
    for validation in instr:
        if ":" in validation:
            instr_dict[validation.split(":")[0]] = validation.split(":")[1]
        else:
            instr_dict["else"] = validation
    
    if len(set(instr_dict.values())) == 1:
        value = next(iter(instr_dict.values()))
        instr_dict = {}
        instr_dict["always"] = value
    return name, instr_dict

def parse_instructions(instructions: list[str]) -> dict:
    instruction_dict = {}
    for instruction in instructions:
        name, simplified = simplify_instruction(instruction=instruction)
        instruction_dict[name] = simplified
    return instruction_dict

def parse_parts(parts: list[str]) -> list[dict]:
    parts_dict_list = []
    for part in parts:
        part_dict = {}
        part_parts = part[1:-1].split(",")
        for part_part in part_parts:
            part_dict[part_part[0]] = part_part[2:]
        parts_dict_list.append(part_dict)
    return parts_dict_list

def traverse_instructions(part_dict: dict, instruction_dict: dict) -> str:
    current_instruction = "in"
    while True:
        instruction_set = instruction_dict[current_instruction]
        # print(f"Working on instruction set: {current_instruction}")
        for k, v in instruction_set.items():
            value = "-1"
            if k == "always":
                return v
            if k == "else":
                value = v
            elif OPS[k[1]](int(part_dict[k[0]]),int(k[2:])):
                value = v
            
            if value != "-1":
                if value == "A" or value == "R":
                    return value
                else:
                    current_instruction = value
                    break


In [5]:
instruction_dict = parse_instructions(instructions=instructions)
part_dicts = parse_parts(parts=parts)

In [6]:
total_sum = 0
for part_dict in part_dicts:
    if traverse_instructions(part_dict=part_dict, instruction_dict=instruction_dict) == "A":
        total_sum += int(part_dict["x"])+int(part_dict["m"])+int(part_dict["a"])+int(part_dict["s"])
print(total_sum)

383682


# Imports pt.2

In [41]:
import re
from collections import deque

# Pt. 2

In [77]:
def different_traverse_instructions(instruction_dict: dict, instruction_index: str, ranges: list[list]) -> list[dict]:
    new_instructions = []
    instruction_set = instruction_dict[instruction_index]
    for instruction, target in instruction_set.items():
        if ">" in instruction or "<" in instruction:
            xmas, op, threshold = re.match("([xmas])([><])(\d+)", instruction).groups()
            filtered_ranges = ranges.copy()
            filtered_ranges["xmas".find(xmas)] = [value for value in filtered_ranges["xmas".find(xmas)] if OPS[op](int(value),int(threshold))]
            ranges["xmas".find(xmas)] = [value for value in ranges["xmas".find(xmas)] if not OPS[op](int(value),int(threshold))]
            new_instructions.append((target, filtered_ranges))
        else:
            new_instructions.append((target, ranges))
    return new_instructions
            

In [83]:
ranges = [[i+1 for i in range(4000)]]*4

total_option_count = 0

instruction_list = deque()
instruction_list.append(("in", ranges,))
while instruction_list:
    next_instruction = instruction_list.popleft()

    if next_instruction[0] == "A":
        range_len_list = [len(x) for x in next_instruction[1]]
        total_option_count += range_len_list[0]*range_len_list[1]*range_len_list[2]*range_len_list[3]
        continue
    if next_instruction[0] == "R":
        continue

    next_steps = different_traverse_instructions(instruction_dict=instruction_dict, instruction_index=next_instruction[0], ranges=next_instruction[1])
    for next_step in next_steps:
        instruction_list.append(next_step)

total_option_count

117954800808317